In [ ]:
%pylab notebook
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!ls data/manuvers_raw/

In [ ]:
import pickle
data=[]
fd=open('data/manuvers_raw/mov4.pkl','rb')
while 1:
    try:
        data.append(pickle.load(fd))
    except EOFError:
        break

In [ ]:
data[600]

In [ ]:
def smaple_reader():
    for d in data:
        yield d
sr=smaple_reader()


In [ ]:
calib_samples=[sr.__next__() for _ in range(100)]

In [ ]:
acc_mat=np.array([a['a/g'][:3] for a in calib_samples if 'a/g' in a])
alt_mat=np.array([a['alt'] for a in calib_samples if 'alt' in a])

In [ ]:
acc_mat

In [ ]:
plt.figure()
plt.subplot(2,1,1)
plt.plot(acc_mat[:,0],'r')
plt.plot(acc_mat[:,1],'g')
plt.plot(acc_mat[:,2],'b')
#g_mean=acc_mat[:,2].mean()
#g_mean=np.sqrt((acc_mat[:,:3]**2).sum(axis=1))#.mean()
g_mean=acc_mat[:,:3].mean(axis=0)

g_unit=g_mean/np.sqrt((g_mean**2).sum())

g_scale=np.sqrt((g_mean**2).sum())
print(g_mean)
plt.subplot(2,1,2)
plt.plot(alt_mat,'r')
alt_var=alt_mat.var()
print(g_unit)

In [ ]:
def get_scaled_g_alt(d):
    #return (np.sqrt((d['a/g'][:3]**2).sum())/g_mean-1)*9.8,d['alt']    
    #return np.sqrt((d['a/g'][:3]**2).sum(axis=0)),d['alt']
    return (d['a/g'][:3]*g_unit).sum()/g_scale-1.0,d['alt']
    #return d['a/g'][2],d['alt']




In [ ]:
###kalman
def kalman(start_alt):
    Xk=np.array([0,0,start_alt]) #acc vel pos
    def F(X,dt):
        pass

In [ ]:
az_scale=[]
alt_vec=[]
t_stemp=[]
t_stemp_internal=[]
while 1:
    try:
        next_data=sr.__next__()
        if 'alt' in next_data:
            az,alt=get_scaled_g_alt(next_data)
            az_scale.append(az)
            alt_vec.append(alt)
            t_stemp.append(next_data['s_sync'])
            t_stemp_internal.append(next_data['t_stemp_ms'])
    except StopIteration:
        break
t_stemp=np.array(t_stemp)
t_stemp-=t_stemp[0]
t_stemp_internal=np.array(t_stemp_internal)
t_stemp_internal-=t_stemp_internal[0]
t_stemp_internal=t_stemp_internal/1000.0


In [ ]:
#print(az_scale)
plt.figure()
ax=plt.subplot(2,1,1)
plt.grid('on')
plt.plot(t_stemp,az_scale,'-+')
plt.plot(t_stemp_internal,az_scale,'-')
plt.subplot(2,1,2,sharex=ax)
plt.plot(t_stemp,alt_vec)
fsize=10
plt.plot(t_stemp[fsize/2:-fsize/2+1],np.convolve(np.ones(fsize)/fsize,alt_vec,'valid'))


In [ ]:
plt.figure()
plot(t_stemp,t_stemp_internal,'+')